In [1]:
import numpy 
import torch
import json
import ast
import pandas as pd
import re

In [2]:
r"""return:
1. inspired_movie_db: ID to descpiton + actor + ...
2. train_dialogs
3. dev_dialogs
4. test_dialogs

"""

'return:\n1. inspired_movie_db: ID to descpiton + actor + ...\n2. train_dialogs\n3. dev_dialogs\n4. test_dialogs\n\n'

In [3]:
# Inspired database

inspired_movie_db = {} # ID to descripiton
movie_to_id = {}

key = 0
df = pd.read_csv("/home/by2299/RedialData/DATA/movie_database.tsv", sep = '\t')
for i in range(len(df)):
    mid = df['imdb_id'][i]
    name = df['title'][i].lower()
    year = df['year'][i]
    desc = df['short_plot'][i]
    full_name = name + ' (' + str(year) + ')'
    
    concated_meta = str(name) + " [SEP] " + \
      str(df['actors'][i]) + " [SEP] " + \
      str(df['director'][i]) + " [SEP] " + \
      str(df['genre'][i]) + " [SEP] " + \
      str(desc)

    # for movie match in inspire dataset
    inspired_movie_db[key] = concated_meta
    movie_to_id[full_name] = key
    key +=1 
print(len(inspired_movie_db))

17869


In [4]:
train_path = "/home/by2299/Full_CRS/Data/Inspired/dialog_data/train.tsv"
dev_path = "/home/by2299/Full_CRS/Data/Inspired/dialog_data/dev2.tsv"
test_path = "/home/by2299/Full_CRS/Data/Inspired/dialog_data/test.tsv"

In [5]:
def collect_dialog(data_path, movie_db):
    
    data = pd.read_csv (data_path, sep = '\t')
    dialog_group = dict()

    for i in range(len(data['dialog_id'])):
        dialog_id = data['dialog_id'][i]
        if dialog_id not in dialog_group.keys():
            dialog_group[dialog_id] = [i]
        else:
            dialog_group[dialog_id].append(i)

    AB_dict = {}
    AB_dict['RECOMMENDER'] = 'A'
    AB_dict['SEEKER'] = 'B'

    all_dialogs = []
    
    count1=0
    count2=0
    non_exist_movies = {}

    for dial_id, utt_list in dialog_group.items():

        this_dialog = []
        j = 0
        while j < len(utt_list):
            mentioned = []
            userID = data['speaker'][utt_list[j]]
            text = data['text'][utt_list[j]]
            text_p = data['text_with_placeholder'][utt_list[j]]
            mentioned_movies = re.findall(r"\[MOVIE_TITLE_\d+]", text_p)

            movie_dict = ast.literal_eval(data['movie_dict'][utt_list[j]])
            genre_dict = ast.literal_eval(data['genre_dict'][utt_list[j]])
            actor_dict = ast.literal_eval(data['actor_dict'][utt_list[j]])

            movie_dict_r = {}
            genre_dict_r = {}
            actor_dict_r = {}

            for key, value in movie_dict.items():
                movie_dict_r[int(value)] = key
            for key, value in genre_dict.items():
                genre_dict_r[int(value)] = key
            for key, value in actor_dict.items():
                actor_dict_r[int(value)] = key

            if len(mentioned_movies) > 0:         
                for item in mentioned_movies:
                    key = int(re.findall(r"\d", item)[0])
                    movie = movie_dict_r[key]
                    if movie.lower() in movie_db:
                        count1+=1
                        mentioned.append(movie_db[movie.lower()])
                        text_p = text_p.replace(item, '[MOVIE_ID]')
                    else:
                        count2+=1   
                        non_exist_movies[movie] = 1

                mentioned_genre = re.findall(r"\[MOVIE_GENRE_\d+]", text_p)
                for item in mentioned_genre:
                    key = int(re.findall(r"\d", item)[0])
                    if key in genre_dict_r.keys():
                        genre = genre_dict_r[key]
                        text_p = text_p.replace(item, genre)

                mentioned_actor = re.findall(r"\[MOVIE_P_ACTOR_\d+]", text_p)
                for item in mentioned_actor:
                    key = int(re.findall(r"\d", item)[0])
                    actor = actor_dict_r[key]
                    text_p = text_p.replace(item, actor)

                text = text_p

            k = 1
            while j + k < len(utt_list) and userID == data['speaker'][utt_list[j+k]]:
                text_new = data['text'][utt_list[j+k]]
                text_p = data['text_with_placeholder'][utt_list[j+k]]
                mentioned_movies = re.findall(r"\[MOVIE_TITLE_\d+]", text_p)

                if len(mentioned_movies) > 0:         
                    for item in mentioned_movies:
                        key = int(re.findall(r"\d", item)[0])
                        if key in movie_dict_r.keys():
                            movie = movie_dict_r[key]
                            
                            if movie.lower() in movie_db:
                                count1+=1
                                mentioned.append(movie_db[movie.lower()])
                                text_p = text_p.replace(item, '[MOVIE_ID]')
                            else:
                                count2+=1   
                                non_exist_movies[movie] = 1

                    mentioned_genre = re.findall(r"\[MOVIE_GENRE_\d+]", text_p)
                    for item in mentioned_genre:
                        key = int(re.findall(r"\d", item)[0])
                        if key in genre_dict_r.keys():
                            genre = genre_dict_r[key]
                            text_p = text_p.replace(item, genre)

                    mentioned_actor = re.findall(r"\[MOVIE_P_ACTOR_\d+]", text_p)
                    for item in mentioned_actor:
                        key = int(re.findall(r"\d", item)[0])
                        if key in actor_dict_r.keys():
                            actor = actor_dict_r[key]
                            text_p = text_p.replace(item, actor)

                    text_new = text_p

                text = text + ' ' + text_new

                k += 1
            j = j + k
            if mentioned == []:
                mentioned = None
            this_dialog.append((AB_dict[userID] + ": " + text, mentioned)) 

        all_dialogs.append(this_dialog)
    
    print(count1, count2)

    return all_dialogs, non_exist_movies

In [6]:
train_dialogs, train_non_exist_movies = collect_dialog(train_path, movie_to_id)
dev_dialogs, dev_non_exist_movies = collect_dialog(dev_path, movie_to_id)
test_dialogs, test_non_exist_movies = collect_dialog(test_path, movie_to_id)

2697 1430
367 175
300 179


In [7]:
all_non_exist_movies = {}
for key in train_non_exist_movies.keys():
    all_non_exist_movies[key] = 1
for key in dev_non_exist_movies.keys():
    all_non_exist_movies[key] = 1
for key in test_non_exist_movies.keys():
    all_non_exist_movies[key] = 1

In [8]:
# IMDb dataset

df = pd.read_csv("/home/by2299/RedialData/IMDb.csv")

# our ID to description
all_movie_db = {} # ID to descripiton
imdb_to_id = {}

# unique imdb id set
imdb_id_set = {}

# unique duplicate name set
duplicate_name_set = {}

key = 0
for i in range(len(df)):
    mid = df['imdb_title_id'][i]
    if mid not in imdb_id_set.keys():
        imdb_id_set[mid]=1
        name = df['original_title'][i].lower()
        
        top3_actors = "nan" if str(df['actors'][i]) == "nan" else ','.join(df['actors'][i].split(',')[:3])
        director  = df['director'][i]
        genres = df['genre'][i]
        
        year = df['year'][i]
        full_name = name + ' (' + str(year) + ')'
        desc = df['description'][i]
        
        concated_meta = str(name) + ' [SEP] ' + str(top3_actors) + ' [SEP] ' + \
            str(director) + ' [SEP] ' + str(genres) + ' [SEP] ' + str(desc)
    
        
        # for movie match in redial dataset
        all_movie_db[key] = concated_meta
        imdb_to_id[full_name] = key
        key +=1 

# Movies_metadata dataset
new_count = 0
df = pd.read_csv("/home/by2299/RedialData/movies_metadata.csv")
for i in range(len(df)):
    mid = df['imdb_id'][i]
    if mid not in imdb_id_set.keys():
        new_count += 1
        imdb_id_set[mid]=1
        name = df['original_title'][i].lower()
        
        genres = json.loads(df["genres"][i].replace('\'', '\"'))
        genres = ', '.join([ g['name'] for g in genres])
        
        year = df['release_date'][i]
        year = re.findall(r"\d\d\d\d", str(year))
        if len(year) > 0:
            year = year[0]
            full_name = name + ' (' + str(year) + ')'
        else:
            full_name = name
        desc = df['overview'][i]
        
        concated_meta = str(name) + ' [SEP] [SEP] [SEP] ' + str(genres) + ' [SEP] ' + str(desc)
                
        # for movie match in redial dataset
        all_movie_db[key] = concated_meta
        imdb_to_id[full_name] = key
        key +=1 

/home/by2299/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/by2299/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
count = 0
last_non_exist_movies = []

key = len(inspired_movie_db)
for i in all_non_exist_movies.keys():
    if i.lower() not in imdb_to_id:
        count += 1
        last_non_exist_movies.append(i)
    else:
        inspired_movie_db[key] = all_movie_db[imdb_to_id[i.lower()]]
        movie_to_id[i.lower()] = key
        key+=1
print(len(last_non_exist_movies))
print(len(inspired_movie_db))

361
18225


In [10]:
from tmdbv3api import TMDb
from tmdbv3api import Movie

tmdb = TMDb()
tmdb.api_key = 'ecbe5b50079424c4372da15d2999da46'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()

In [11]:
def meta_with_id(m_id, movie_name_no_year):
    movie_deets = movie.details(m_id)
    
    #actors
    actors = ''
    for i, cast in enumerate(movie_deets['casts']['cast']):
        if i <= 1:
            actors += cast['name'] + ', '
        else:
            actors += cast['name']
            break
    
    # genre
    genres = ''
    for i, genre in enumerate(movie_deets["genres"]):
        genres += genre['name'] + ', '
        
    # overview
    overview = movie_deets['overview']
    
    return movie_name_no_year + ' [SEP] ' + actors + ' [SEP] [SEP] ' + genres + ' [SEP] ' + overview

In [12]:
still_unfound_sad = []
key = len(inspired_movie_db)
for movie_name_unfound in last_non_exist_movies:
    movie_name_unfound_no_year = movie_name_unfound
    if '(' in movie_name_unfound:
        movie_name_unfound_no_year = ' '.join(movie_name_unfound.split(' ')[:-1])
    if movie_name_unfound_no_year == '': continue
    results = movie.search(movie_name_unfound_no_year)
    if len(results) == 0: still_unfound_sad.append(movie_name_unfound); continue
    if len(results) == 1:
        inspired_movie_db[key] = meta_with_id(results[0]['id'], movie_name_unfound_no_year)
        movie_to_id[movie_name_unfound.lower()] = key
#         imdb_name_without_year_to_id_full[movie_name_unfound_no_year] = key
        key += 1
        continue
    found = False
    for result in results:
        o_title = result['original_title']
        title = result['title']
        if o_title.lower() == movie_name_unfound_no_year.lower() or title.lower() == movie_name_unfound_no_year.lower():
            inspired_movie_db[key] = meta_with_id(result['id'], movie_name_unfound_no_year)
            movie_to_id[movie_name_unfound.lower()] = key
#             imdb_name_without_year_to_id_full[movie_name_unfound_no_year] = key
            key += 1
            found = True
            break
    if not found:
        still_unfound_sad.append(movie_name_unfound)
    
print(len(still_unfound_sad))
print(len(inspired_movie_db))

116
18469


In [13]:
train_dialogs, train_non_exist_movies = collect_dialog(train_path, movie_to_id)
dev_dialogs, dev_non_exist_movies = collect_dialog(dev_path, movie_to_id)
test_dialogs, test_non_exist_movies = collect_dialog(test_path, movie_to_id)

3754 373
517 25
431 48


In [14]:
train_dialogs[0]

[('A: Hi There! What types of movies do you like to watch?', None),
 ("B: Hello! I'm more of an action movie or a good romance and mystery movie.",
  None),
 ('A: I just saw the trailer for [MOVIE_ID] when I went to see Joker and it looked like a good mix of action and',
  [15]),
 ('B: I seen that one too as I seen [MOVIE_ID] about a month ago. I thought about asking my fiance about going and seeing it.',
  [45]),
 ('A: It looks like a good movie for people who like many different movies. It also has a great cast! I was surprised to see Chris Evans in the trailer!',
  None),
 ("B: Maybe with Chris Evans in it it'll be easier to convince my fiance to see it. Do you know who else is in the cast?",
  None),
 ('A: Daniel Craig and Jamie Lee Curtis are also in the cast. Daniel Craig does a lot of 007 so definitely a good hearthrob role to convince the misses lol!',
  None),
 ('B: [MOVIE_ID] But he loves the bond movies so that should be a good incentive for him to go see it. Do you have any

In [15]:
inspired_movie_db[8300]

'fragments [SEP] Kate Beckinsale, Forest Whitaker, Guy Pearce, Dakota Fanning [SEP] Rowan Woods [SEP] Crime, Drama [SEP] A group of strangers form a unique relationship with each other after surviving a random shooting at a Los Angeles diner.'

In [16]:
torch.save(train_dialogs, "/local-scratch1/data/by2299/inspired_train_placeholder")
torch.save(dev_dialogs, "/local-scratch1/data/by2299/inspired_dev_placeholder")
torch.save(test_dialogs, "/local-scratch1/data/by2299/inspired_test_placeholder")

torch.save(inspired_movie_db, "/local-scratch1/data/by2299/inspired_movie_db")